In [1]:

using ITensors
using ITensors: position!
#using ITensorTDVP
using KrylovKit
using Printf
#using ProgressMeter
#ProgressMeter.ijulia_behavior(:append)

# Simple TDVP

Below is a code for a simple TDVP implementation (both for 1-site & 2-site variant). This is based on the ITensorTDVP. The function tdvp! takes the folllowing arguments.

**Initial wavefunction** <br>
**Hamiltonian :** (in MPO form) to time evolve the wave function<br>
**dt(time step)** <br>
**tf(final time )** : time upto which you want to evolve the system)<br>
**reverse_step_val :** Decides whether you need the right-to-left swap (Changes the accuracy O(dt^3) for true,O(dt^2) for no)

Time is entered as real for time evolution. The i is multiplied internally.

In [5]:
struct TDVP end
set_nsite!(PH::ProjMPO,nsite) = (PH.nsite = nsite)

function solver(H, t, psi0; kwargs...)
    solver_kwargs = (;
      ishermitian=get(kwargs, :ishermitian, true),
      issymmetric=get(kwargs, :issymmetric, true),
      tol=get(kwargs, :solver_tol, 1E-12),
      krylovdim=get(kwargs, :solver_krylovdim, 30),
      maxiter=get(kwargs, :solver_maxiter, 100),
      verbosity=get(kwargs, :solver_outputlevel, 0),
      eager=true,
    )
    psi, info = exponentiate(H, t, psi0; solver_kwargs...)
    return psi, info
end

function measure_Sz(psi,n)
    psi = ITensors.orthogonalize(psi,n)
    sn = siteind(psi,n)
    Sz = scalar(dag(prime(psi[n],"Site"))*op("Sz",sn)*psi[n])
    return real(Sz)
end


function tdvp!(ψ::MPS,H::MPO,dt::Float64,tf::Float64,nsite::Int64,reverse_step_val; kwargs...)
    
    num_time_steps = Int(tf/dt)
    
    #kwargs
   
    #The error tolerance for `KrylovKit.exponentiate`
    exp_tol = get(kwargs,:exp_tol, 1e-14)
    #Passed to KrylovKit
    krylovdim = get(kwargs,:krylovdim, 30 )
    maxiter = get(kwargs,:maxiter,100)
    normalize = get(kwargs,:normalize,true)
    #Is the Hamiltonian H hermitian? Useful to KrylovKit for exponentiation
    hermitian = get(kwargs,:hermitian,true)
    
    maxdim::Integer = get(kwargs, :maxdim, typemax(Int))
    mindim::Integer = get(kwargs, :mindim, 1)
    cutoff::Real = get(kwargs, :cutoff, 1E-16)
    
    #If progress bar is used
    #pbar = get(kwargs,:progress, true) ? Progress(num_time_steps, desc="Evolving state... ") : nothing
    
   #Imaginary time step
    τ = 1im*dt
    
    N = length(ψ)
    #start with right orthogonalised wave function
    ITensors.orthogonalize!(ψ,1)
    
    #Projected MPO
    #https://docs.juliahub.com/ITensors/P3pqL/0.1.7/ProjMPO.html
    
    PH = ProjMPO(H)
    position!(PH,ψ,1)
    
    
    
    set_nsite!(PH, nsite)
    
    
    maxtruncerr = 0.0
    
    
    #Do the time evolution
    for time_step in 1:num_time_steps
        stime  = @elapsed begin
        # b is the active site  ha gives 1/2 depending on right/left swap
        current_time = dt*time_step
        for (b,ha) in sweepnext(N; ncenter=nsite)
                # Do 'forwards' evolution step
                
                
                set_nsite!(PH, nsite)
                #orthogonalise at position b
                
                position!(PH, ψ, b)
                if nsite == 1
                  #println(" Forward phi1 Selected")
                  phi1 = ψ[b]
                elseif nsite == 2
                  phi1 = ψ[b] * ψ[b + 1]
                end

                
                #orthogonalise at position b
                
                #phi1, info = exponentiate(PH, -τ/2, phi1; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                phi1, info = solver(PH, -τ/2, phi1)
                info.converged==0 && throw("exponentiate did not converge")
                
                spec = nothing
                
                
                if nsite == 1
                  ψ[b] = phi1
                  Sz = measure_Sz(ψ,b) 
                  #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
                  flush(stdout)
                  #println(" Forward phi1 Replaced")
                elseif nsite == 2
                  #Determine sweep direction
                  dir = ha == 1 ? "left" : "right"
                  spec = replacebond!(ψ,b,phi1;mindim,cutoff,normalize=normalize, ortho = dir, kwargs... )
                  maxtruncerr = max(maxtruncerr, spec.truncerr)
                end
                
                if(reverse_step_val == true)
                    if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))


                        b1 = (ha==1 ? b + 1 : b)
                        Δ = (ha==1 ? +1 : -1)

                        if nsite == 2
                            phi0 = ψ[b1]
                        elseif nsite == 1
                            #println(" SVD")
                            uinds = uniqueinds(phi1, ψ[b + Δ])
                            U, S, V = svd(phi1, uinds)
                            ψ[b] = U
                            phi0 = S * V
                            if ha == 1
                              ITensors.setleftlim!(ψ, b)
                            elseif ha == 2
                              ITensors.setrightlim!(ψ, b)
                            end
                        end

                        # evolve with single-site Hamiltonian backward in time.
                        # In the case of imaginary time-evolution this step
                        # is not necessary (see Ref. [1])




                        set_nsite!(PH, nsite - 1)
                        ITensors.position!(PH, ψ, b1)
                        phi0, info = exponentiate(PH, τ/2, phi0; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                        normalize && (phi0 ./= norm(phi0))
                        info.converged==0 && throw("exponentiate did not converge")


                        if nsite == 2
                            ψ[b1] = phi0
                            #print(" Reverse phi  calculated")
                        elseif nsite == 1
                            #println(" Reverse phi  calculated")
                            ψ[b + Δ] = phi0 * ψ[b + Δ]
                            if ha == 1
                              ITensors.setrightlim!(ψ, b + Δ + 1)
                            elseif ha == 2
                              ITensors.setleftlim!(ψ, b + Δ - 1)
                            end
                        end

                        set_nsite!(PH, nsite)



                    end
                    #if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))
                end
                #for if(reverse_step_val == true)
        end
        #for(b,ha)
            
            #Sz = measure_Sz(ψ,5) 
            #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
            flush(stdout)

        
        end
        #time elapsed
        
        
       
        
    end
    #External for loop for time_step
end
#function


tdvp! (generic function with 2 methods)

In [6]:
N = 15

s = siteinds("S=1/2", N)

os = OpSum()
for j in 1:(N - 1)
    os += 0.5, "S+", j, "S-", j + 1
    os += 0.5, "S-", j, "S+", j + 1
    os += "Sz", j, "Sz", j + 1
end

H = MPO(os, s)
#ψ = productMPS(s, n -> isodd(n) ? "Up" : "Dn")
ψ=randomMPS(s; linkdims=5)
#psi = productMPS(s, n -> isodd(n) ? "Up" : "Dn")


for j=1:N
    println("site : ",j," ",measure_Sz(ψ,j))
end

ψ4 = deepcopy(ψ)
ψ5 = deepcopy(ψ)

site : 1 -0.08526870311549709
site : 2 -0.08851297246585628
site : 3 -0.07673160975559071
site : 4 -0.09912713033375235
site : 5 0.0034443860263250192
site : 6 -0.10520860836736742
site : 7 -0.04403816787634869
site : 8 -0.09786454743626884
site : 9 -0.15922833447200693
site : 10 -0.07259608020038817
site : 11 -0.012795945457025516
site : 12 -0.11286909462130937
site : 13 0.12717016135708845
site : 14 -0.09544193969514775
site : 15 0.015652922504292753


MPS
[1] ((dim=2|id=931|"S=1/2,Site,n=1"), (dim=5|id=67|"Link,l=1"))
[2] ((dim=5|id=67|"Link,l=1"), (dim=2|id=335|"S=1/2,Site,n=2"), (dim=5|id=931|"Link,l=2"))
[3] ((dim=5|id=931|"Link,l=2"), (dim=2|id=845|"S=1/2,Site,n=3"), (dim=5|id=130|"Link,l=3"))
[4] ((dim=5|id=130|"Link,l=3"), (dim=2|id=375|"S=1/2,Site,n=4"), (dim=5|id=414|"Link,l=4"))
[5] ((dim=5|id=414|"Link,l=4"), (dim=2|id=908|"S=1/2,Site,n=5"), (dim=5|id=656|"Link,l=5"))
[6] ((dim=5|id=656|"Link,l=5"), (dim=2|id=827|"S=1/2,Site,n=6"), (dim=5|id=726|"Link,l=6"))
[7] ((dim=5|id=726|"Link,l=6"), (dim=2|id=488|"S=1/2,Site,n=7"), (dim=5|id=686|"Link,l=7"))
[8] ((dim=5|id=686|"Link,l=7"), (dim=2|id=400|"S=1/2,Site,n=8"), (dim=5|id=796|"Link,l=8"))
[9] ((dim=5|id=796|"Link,l=8"), (dim=2|id=586|"S=1/2,Site,n=9"), (dim=5|id=365|"Link,l=9"))
[10] ((dim=5|id=365|"Link,l=9"), (dim=2|id=873|"S=1/2,Site,n=10"), (dim=5|id=475|"Link,l=10"))
[11] ((dim=5|id=475|"Link,l=10"), (dim=2|id=41|"S=1/2,Site,n=11"), (dim=5|id=23|"Link,l=11"))
[12] ((d

In [11]:
ψ1 = deepcopy(ψ)
#2-site TDVP
tdvp!(ψ1,H,0.05,0.5,2,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.
for j=1:N
    println("site : ",j," ",measure_Sz(ψ1,j))
end



site : 1 -0.093452863185101
site : 2 -0.08178489398528875
site : 3 -0.0794771425311438
site : 4 -0.08819919115203972
site : 5 -0.011680587017789441
site : 6 -0.09106692935222331
site : 7 -0.0510131358764921
site : 8 -0.1024706608447248
site : 9 -0.15432850164863599
site : 10 -0.07113860870382355
site : 11 -0.01850252707471057
site : 12 -0.08833401903445237
site : 13 0.09584451193573146
site : 14 -0.06397973967394525
site : 15 -0.0038313774334711864


In [12]:
ψ2 = deepcopy(ψ)

#1-site TDVP
tdvp!(ψ2,H,0.05,0.5,1,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.

for j=1:N
    println("site : ",j," ",measure_Sz(ψ2,j))
end



site : 1 -0.0934596755567175
site : 2 -0.08178304607803671
site : 3 -0.07999076913150935
site : 4 -0.09015637803826905
site : 5 -0.004239550097459133
site : 6 -0.09799494904165015
site : 7 -0.04746173490546908
site : 8 -0.10401669117613827
site : 9 -0.1564098829520334
site : 10 -0.07184855506132783
site : 11 -0.011227147423697383
site : 12 -0.09378763327426859
site : 13 0.09676414154146194
site : 14 -0.06396636073371388
site : 15 -0.0038319107217482523


In [13]:
ψ3 = deepcopy(ψ)
cutoff = 1e-15

ttotal = 0.5
tau = 0.05
current_time = 0
Nsteps = Int(ttotal/tau)

gates = ITensor[]
  for j=1:N-1
    s1 = s[j]
    s2 = s[j+1]
    hj =       op("Sz",s1) * op("Sz",s2) +
         1/2 * op("S+",s1) * op("S-",s2) +
         1/2 * op("S-",s1) * op("S+",s2)
    Gj = exp(-1.0im * tau/2 * hj)
    push!(gates,Gj)
  end


append!(gates,reverse(gates))



for step=1:Nsteps
    psi = apply(gates, ψ3; cutoff=cutoff)
    current_time += tau
    Sz = measure_Sz(ψ3,5)
    #println("$current_time   $Sz")
end


#Calculate magnetisation at all sites.

for j=1:N
    println("site : ",j," ",measure_Sz(ψ3,j))
end




site : 1 -0.08526870311549709
site : 2 -0.08851297246585628
site : 3 -0.07673160975559071
site : 4 -0.09912713033375235
site : 5 0.0034443860263250192
site : 6 -0.10520860836736742
site : 7 -0.04403816787634869
site : 8 -0.09786454743626884
site : 9 -0.15922833447200693
site : 10 -0.07259608020038817
site : 11 -0.012795945457025516
site : 12 -0.11286909462130937
site : 13 0.12717016135708845
site : 14 -0.09544193969514775
site : 15 0.015652922504292753


In [82]:
ψ4 = deepcopy(ψ)

ψ4 = tdvp(
  H,
  -1.0*1im,
  ψ4;
  nsweeps=5,
  reverse_step=true,
  normalize=true,
  maxdim=30,
  cutoff=1e-10,
  outputlevel=1,
  nsite = 2
) 

After sweep 1: maxlinkdim=20 maxerr=1.01E-16 current_time=0.0 - 1.0im time=0.101
After sweep 2: maxlinkdim=30 maxerr=7.14E-06 current_time=0.0 - 2.0im time=0.219
After sweep 3: maxlinkdim=30 maxerr=2.88E-04 current_time=0.0 - 3.0im time=0.232
After sweep 4: maxlinkdim=30 maxerr=2.04E-03 current_time=0.0 - 4.0im time=0.233
After sweep 5: maxlinkdim=30 maxerr=5.18E-03 current_time=0.0 - 5.0im time=0.255


MPS
[1] ((dim=2|id=2|"Link,l=1"), (dim=2|id=450|"S=1/2,Site,n=1"))
[2] ((dim=4|id=129|"Link,l=2"), (dim=2|id=681|"S=1/2,Site,n=2"), (dim=2|id=2|"Link,l=1"))
[3] ((dim=8|id=124|"Link,l=3"), (dim=2|id=982|"S=1/2,Site,n=3"), (dim=4|id=129|"Link,l=2"))
[4] ((dim=16|id=37|"Link,l=4"), (dim=2|id=818|"S=1/2,Site,n=4"), (dim=8|id=124|"Link,l=3"))
[5] ((dim=30|id=198|"Link,l=5"), (dim=2|id=729|"S=1/2,Site,n=5"), (dim=16|id=37|"Link,l=4"))
[6] ((dim=30|id=995|"Link,l=6"), (dim=2|id=642|"S=1/2,Site,n=6"), (dim=30|id=198|"Link,l=5"))
[7] ((dim=30|id=771|"Link,l=7"), (dim=2|id=859|"S=1/2,Site,n=7"), (dim=30|id=995|"Link,l=6"))
[8] ((dim=30|id=74|"Link,l=8"), (dim=2|id=348|"S=1/2,Site,n=8"), (dim=30|id=771|"Link,l=7"))
[9] ((dim=30|id=46|"Link,l=9"), (dim=2|id=64|"S=1/2,Site,n=9"), (dim=30|id=74|"Link,l=8"))
[10] ((dim=30|id=36|"Link,l=10"), (dim=2|id=658|"S=1/2,Site,n=10"), (dim=30|id=46|"Link,l=9"))
[11] ((dim=16|id=520|"Link,l=11"), (dim=2|id=48|"S=1/2,Site,n=11"), (dim=30|id=36|"Link,l=10"))
[12

In [83]:
for j=1:N
    println("site : ",j," ",measure_Sz(ψ4,j))
end



site : 1 -0.0864515260159397
site : 2 -0.1372697747021163
site : 3 -0.10306718853261407
site : 4 -0.11445201371170335
site : 5 -0.11007629741898352
site : 6 -0.07156634942917162
site : 7 -0.07059097350693065
site : 8 -0.030780431525036265
site : 9 -0.0403147463875579
site : 10 -0.03128804023562393
site : 11 -0.02716403028438104
site : 12 -0.016209932134029992
site : 13 -0.07488753328380864
site : 14 -0.060685216462866516
site : 15 -0.002882363495958107


In [72]:
ψ5 = deepcopy(ψ)

ψ5 = tdvp(
  H,
  -1.0*1im,
  ψ4;
  nsweeps=5,
  reverse_step=true,
  normalize=true,
  maxdim=30,
  cutoff=1e-10,
  outputlevel=1,
  nsite = 1
) 

After sweep 1: maxlinkdim=30 maxerr=0.00E+00 current_time=0.0 - 1.0im time=0.386
After sweep 2: maxlinkdim=30 maxerr=0.00E+00 current_time=0.0 - 2.0im time=0.122
After sweep 3: maxlinkdim=30 maxerr=0.00E+00 current_time=0.0 - 3.0im time=0.115
After sweep 4: maxlinkdim=30 maxerr=0.00E+00 current_time=0.0 - 4.0im time=0.113
After sweep 5: maxlinkdim=30 maxerr=0.00E+00 current_time=0.0 - 5.0im time=0.113


MPS
[1] ((dim=2|id=291|"Link,u"), (dim=2|id=993|"S=1/2,Site,n=1"))
[2] ((dim=4|id=841|"Link,u"), (dim=2|id=617|"S=1/2,Site,n=2"), (dim=2|id=291|"Link,u"))
[3] ((dim=8|id=831|"Link,u"), (dim=2|id=904|"S=1/2,Site,n=3"), (dim=4|id=841|"Link,u"))
[4] ((dim=16|id=561|"Link,u"), (dim=2|id=232|"S=1/2,Site,n=4"), (dim=8|id=831|"Link,u"))
[5] ((dim=30|id=501|"Link,u"), (dim=2|id=162|"S=1/2,Site,n=5"), (dim=16|id=561|"Link,u"))
[6] ((dim=30|id=196|"Link,u"), (dim=2|id=694|"S=1/2,Site,n=6"), (dim=30|id=501|"Link,u"))
[7] ((dim=30|id=433|"Link,u"), (dim=2|id=499|"S=1/2,Site,n=7"), (dim=30|id=196|"Link,u"))
[8] ((dim=30|id=599|"Link,u"), (dim=2|id=487|"S=1/2,Site,n=8"), (dim=30|id=433|"Link,u"))
[9] ((dim=30|id=629|"Link,u"), (dim=2|id=668|"S=1/2,Site,n=9"), (dim=30|id=599|"Link,u"))
[10] ((dim=30|id=551|"Link,u"), (dim=2|id=173|"S=1/2,Site,n=10"), (dim=30|id=629|"Link,u"))
[11] ((dim=16|id=896|"Link,u"), (dim=2|id=169|"S=1/2,Site,n=11"), (dim=30|id=551|"Link,u"))
[12] ((dim=8|id=892|"Link,u"), (di

In [73]:
for j=1:N
    println("site : ",j," ",measure_Sz(ψ5,j))
end

site : 1 0.03860109646787632
site : 2 0.034605718964978174
site : 3 0.04510344565698165
site : 4 0.00964804029571964
site : 5 0.023088110795177037
site : 6 0.02568038288648089
site : 7 -0.00408984113220073
site : 8 0.008745123586907556
site : 9 -0.04504117232291012
site : 10 -0.0426517178367074
site : 11 -0.0786536857155359
site : 12 -0.1084536247008053
site : 13 -0.057703645361959294
site : 14 -0.09079304845974313
site : 15 -0.0769379783483124
